In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import xlrd 
import json


# # Import API key
# from api_keys import api_key


In [3]:
#import data from url to get the state abbr
url="http://worldpopulationreview.com/static/states/name-abbr.json"
d = requests.get(url).json()

states_abbr = pd.DataFrame(d.items() , columns=["state","abbr"])
states_abbr.head()

,state,abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [4]:
#import excel file with obesity, unemployment, education
xls = pd.ExcelFile('Resources/2018 County Health Rankings Data - v2.xls')
df = pd.read_excel(xls, 'Ranked Measure Data', skiprows=1)


In [5]:
df.head()


,FIPS,State,County,Years of Potential Life Lost Rate,95% CI - Low,95% CI - High,Quartile,Years of Potential Life Lost Rate (Black),Years of Potential Life Lost Rate (Hispanic),Years of Potential Life Lost Rate (White),...,95% CI - High.20,Quartile.33,% Drive Alone (Black),% Drive Alone (Hispanic),% Drive Alone (White),# Workers who Drive Alone,% Long Commute - Drives Alone,95% CI - Low.21,95% CI - High.21,Quartile.34
0,1001.0,Alabama,Autauga,9409.294766,8492.111635,10326.477898,2,11806.223285,NaN,9148.381743,...,88.431482,4,82.494851,NaN,82.539683,21082.0,40.8,36.698437,44.901563,3
1,1003.0,Alabama,Baldwin,7467.596664,7025.900408,7909.292920,1,10335.702691,2827.348216,7484.117065,...,86.017804,2,85.567010,68.864800,84.646255,73058.0,40.1,37.614952,42.585048,3
2,1005.0,Alabama,Barbour,8929.474982,7633.929691,10225.020273,1,9141.333085,NaN,8896.000532,...,85.796041,1,80.341227,NaN,85.656228,7271.0,35.1,29.740946,40.459054,2
3,1007.0,Alabama,Bibb,11741.938889,10057.801626,13426.076153,4,11297.166352,NaN,11923.067966,...,89.905820,2,NaN,NaN,NaN,7044.0,48.8,40.894758,56.705242,4
4,1009.0,Alabama,Blount,9359.122343,8463.490290,10254.754396,1,NaN,5133.144570,9600.725685,...,87.622499,3,NaN,79.539642,81.580997,18384.0,60.2,55.818947,64.581053,4


In [6]:
stats_df = pd.merge(states_abbr, df,
                                 how='left', on='State')

stats_df.head()

KeyError: 'State'

In [15]:
#rename columns used for analysis
df.rename(columns={ df.columns[114]: "Unemployment Percent" }, inplace = True)
df.rename(columns={ df.columns[34]: "Obesity Percent" }, inplace = True)
df.rename(columns={ df.columns[108]: "Some College Percent" }, inplace = True)
df.rename(columns={ df.columns[104]: "High School Grad Percent" }, inplace = True)

Unemployment_Percent=df["Unemployment Percent"]
Obesity_Percent=df["Obesity Percent"]
Some_College=df["Some College Percent"]
High_School=df["High School Grad Percent"]
State_Name=df["State"]
County=df["County"]
state_abbr=states_abbr["abbr"]

#create new Dataframe with just information needed
stats_df= pd.DataFrame({"State": State_Name,
                        "State Abbr": state_abbr,
                        "County": County,
                        "Unemployment (%)": Unemployment_Percent,
                        "Obesity (%)": Obesity_Percent,
                        "Some College (%)": Some_College,
                        "High School Graduate (%)": High_School
                       })

stats_df.head()

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%)
0,Alabama,AL,Autauga,5.271161,36.4,61.942019,90.000000
1,Alabama,AK,Baldwin,5.415263,29.3,63.757100,85.000000
2,Alabama,AS,Barbour,8.648518,44.2,39.723661,86.173469
3,Alabama,AZ,Bibb,6.637117,38.4,49.821371,85.000000
4,Alabama,AR,Blount,5.520897,35.8,53.866457,91.206250


In [16]:
income_xl = pd.ExcelFile('Resources/county income level PROJECT.xlsx')
income_df = pd.read_excel(income_xl, skiprows = [1,2,3,4,5,6,])
income_df.head()

,Unnamed: 0,"Table 1. Per Capita Personal Income by County, 2015 - 2017",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Alabama,Autauga,38591.0,39509.0,40484.0,9,2.4,2.5,67
1,Alabama,Baldwin,41412.0,43004.0,44079.0,4,3.8,2.5,66
2,Alabama,Barbour,31509.0,31750.0,33453.0,46,0.8,5.4,11
3,Alabama,Bibb,29048.0,28989.0,30022.0,66,-0.2,3.6,47
4,Alabama,Blount,31987.0,32261.0,33707.0,45,0.9,4.5,19


In [17]:
income_df.rename(columns={ income_df.columns[2]: "2015($)" }, inplace = True)
income_df.rename(columns={ income_df.columns[3]: "2016($)" }, inplace = True)
income_df.rename(columns={ income_df.columns[4]: "2017($)" }, inplace = True)
income_df.rename(columns = {income_df.columns[0]: "State"}, inplace = True)
income_df.rename(columns = {income_df.columns[1]: "County"}, inplace = True)

income_df.head()

,State,County,2015($),2016($),2017($),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Alabama,Autauga,38591.0,39509.0,40484.0,9,2.4,2.5,67
1,Alabama,Baldwin,41412.0,43004.0,44079.0,4,3.8,2.5,66
2,Alabama,Barbour,31509.0,31750.0,33453.0,46,0.8,5.4,11
3,Alabama,Bibb,29048.0,28989.0,30022.0,66,-0.2,3.6,47
4,Alabama,Blount,31987.0,32261.0,33707.0,45,0.9,4.5,19


In [18]:
year2015_data = income_df["2015($)"]
year2016_data = income_df["2016($)"]
year2017_data = income_df["2017($)"]

income_state_name=income_df["State"]
income_county=income_df["County"]


In [19]:
rename_income_df= pd.DataFrame({"State": income_state_name,
                        "County": income_county,
                        "2015($)": year2015_data,
                        "2016($)": year2016_data,
                        "2017($)": year2017_data,
                       })
rename_income_df.head()

,State,County,2015($),2016($),2017($)
0,Alabama,Autauga,38591.0,39509.0,40484.0
1,Alabama,Baldwin,41412.0,43004.0,44079.0
2,Alabama,Barbour,31509.0,31750.0,33453.0
3,Alabama,Bibb,29048.0,28989.0,30022.0
4,Alabama,Blount,31987.0,32261.0,33707.0


In [23]:
merge_states_info = pd.merge(stats_df, rename_income_df, 
                             right_on=["State","County"], left_on = ["State","County"])
merge_states_info

,State,State Abbr,County,Unemployment (%),Obesity (%),Some College (%),High School Graduate (%),2015($),2016($),2017($)
0,Alabama,AL,Autauga,5.271161,36.4,61.942019,90.000000,38591.0,39509.0,40484.0
1,Alabama,AK,Baldwin,5.415263,29.3,63.757100,85.000000,41412.0,43004.0,44079.0
2,Alabama,AS,Barbour,8.648518,44.2,39.723661,86.173469,31509.0,31750.0,33453.0
3,Alabama,AZ,Bibb,6.637117,38.4,49.821371,85.000000,29048.0,28989.0,30022.0
4,Alabama,AR,Blount,5.520897,35.8,53.866457,91.206250,31987.0,32261.0,33707.0
5,Alabama,CA,Bullock,7.171066,40.2,33.716075,87.500000,26222.0,26450.0,27500.0
6,Alabama,CO,Butler,7.128778,36.0,42.221767,83.000000,33700.0,33888.0,35196.0
7,Alabama,CT,Calhoun,6.697384,36.1,56.743524,92.163374,34188.0,34869.0,36255.0
8,Alabama,DE,Chambers,5.543809,38.0,46.389824,85.000000,31438.0,32562.0,33758.0
9,Alabama,DC,Cherokee,5.213652,35.5,55.565627,88.000000,32906.0,32758.0,34371.0


In [ ]:
#drop na 